In [24]:
import pandas as pd

In [5]:
pd.set_option('display.max_columns', None)

In [13]:
import requests
import pandas as pd
import json 

url = "https://data.cityofchicago.org/resource/9hwr-2zxp.json"
total_rows = 60000
chunk_size = 1000
offset = 0
dataframes = []

while offset < total_rows:
    params = {'$limit': chunk_size, '$offset': offset}
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        if len(data) == 0:  
            break
        df = pd.DataFrame(data)
        dataframes.append(df)
        offset += chunk_size
    else:
        print("Failed to retrieve data. Status code:", response.status_code)
        break

final_df = pd.concat(dataframes, ignore_index=True)

print("Total rows retrieved:", len(final_df))

Total rows retrieved: 60000


In [8]:
import pandas as pd

# Set the max_colwidth option to None to display entire contents of 'location' column
pd.set_option('display.max_colwidth', None)


In [3]:
final_df.dtypes

date                    object
location                object
district                object
block                   object
y_coordinate            object
latitude                object
description             object
location_description    object
updated_on              object
community_area          object
iucr                    object
x_coordinate            object
ward                    object
case_number             object
year                    object
domestic                  bool
fbi_code                object
longitude               object
beat                    object
primary_type            object
arrest                    bool
id                      object
dtype: object

In [20]:
final_df = final_df.drop('location', axis=1)


In [37]:
all_properties.dtypes

property_url      object
mls               object
mls_id            object
status            object
style             object
street            object
unit              object
city              object
state             object
zip_code          object
beds              object
full_baths        object
half_baths        object
sqft              object
year_built        object
days_on_mls       object
list_price        object
list_date         object
sold_price         int64
last_sold_date    object
lot_sqft          object
price_per_sqft    object
latitude          object
longitude         object
stories           object
hoa_fee           object
parking_garage    object
primary_photo     object
alt_photos        object
dtype: object

In [28]:
from homeharvest import scrape_property
from datetime import datetime, timedelta

properties_list = []
start_date = datetime.now() - timedelta(days=500)
end_date = datetime.now()
date_range = (end_date - start_date).days
segments = 90  # Adjust the number of segments as necessary
days_per_request = date_range // segments

for i in range(segments):
    date_from = start_date + timedelta(days=i * days_per_request)
    date_to = date_from + timedelta(days=days_per_request)

    partial_properties = scrape_property(
        location="Chicago, IL",
        listing_type="sold",
        date_from=date_from.strftime("%Y-%m-%d"),
        date_to=date_to.strftime("%Y-%m-%d")
    )

    print(f"Segment {i+1}, Dates: {date_from.strftime('%Y-%m-%d')} to {date_to.strftime('%Y-%m-%d')}, Number of properties: {len(partial_properties)}")
    properties_list.append(partial_properties)

all_properties = pd.concat(properties_list, ignore_index=True)
print(f"Total Number of properties: {len(all_properties)}")

filename = "HomeHarvest_Chicago.csv"
all_properties.to_csv(filename, index=False)


Segment 1, Dates: 2022-08-24 to 2022-08-29, Number of properties: 706
Segment 2, Dates: 2022-08-29 to 2022-09-03, Number of properties: 995
Segment 3, Dates: 2022-09-03 to 2022-09-08, Number of properties: 510
Segment 4, Dates: 2022-09-08 to 2022-09-13, Number of properties: 614
Segment 5, Dates: 2022-09-13 to 2022-09-18, Number of properties: 658
Segment 6, Dates: 2022-09-18 to 2022-09-23, Number of properties: 501
Segment 7, Dates: 2022-09-23 to 2022-09-28, Number of properties: 406
Segment 8, Dates: 2022-09-28 to 2022-10-03, Number of properties: 653
Segment 9, Dates: 2022-10-03 to 2022-10-08, Number of properties: 690
Segment 10, Dates: 2022-10-08 to 2022-10-13, Number of properties: 441
Segment 11, Dates: 2022-10-13 to 2022-10-18, Number of properties: 584
Segment 12, Dates: 2022-10-18 to 2022-10-23, Number of properties: 499
Segment 13, Dates: 2022-10-23 to 2022-10-28, Number of properties: 635
Segment 14, Dates: 2022-10-28 to 2022-11-02, Number of properties: 692
Segment 15, Dat

In [23]:
import sqlite3

In [24]:
# Create a new SQLite database file
conn = sqlite3.connect('chicago-crime-property.db')
cursor = conn.cursor()


In [25]:
# Create a table for the Property data
cursor.execute('''CREATE TABLE Property (
                 property_url TEXT,
                 mls TEXT,
                 mls_id INTEGER,
                 status TEXT,
                 street TEXT,
                 unit TEXT,
                 city TEXT,
                 state TEXT,
                 zip_code TEXT,
                 style TEXT,
                 beds INTEGER,
                 full_baths INTEGER,
                 half_baths INTEGER,
                 sqft INTEGER,
                 year_built INTEGER,
                 stories INTEGER,
                 lot_sqft INTEGER,
                 days_on_mls INTEGER,
                 list_price REAL,
                 list_date TEXT,
                 sold_price REAL,
                 last_sold_date TEXT,
                 price_per_sqft REAL,
                 hoa_fee REAL,
                 latitude REAL,
                 longitude REAL
                 )''')

# Create a table for the Crime data
cursor.execute('''CREATE TABLE Crime (
                 ID INTEGER PRIMARY KEY,
                 CaseNumber TEXT,
                 Date TEXT,
                 Block TEXT,
                 IUCR TEXT,
                 PrimaryType TEXT,
                 Description TEXT,
                 LocationDescription TEXT,
                 Arrest BOOLEAN,
                 Domestic BOOLEAN,
                 Beat INTEGER,
                 District INTEGER,
                 Ward INTEGER,
                 CommunityArea INTEGER,
                 FBICode TEXT,
                 XCoordinate REAL,
                 YCoordinate REAL,
                 Year INTEGER,
                 UpdatedOn TEXT,
                 Latitude REAL,
                 Longitude REAL,
                 Location TEXT
                 )''')

# Commit changes to the database
conn.commit()


OperationalError: table Property already exists

In [26]:
# Convert data types for Crime data
final_df['district'] = final_df['district'].astype(int)
final_df['y_coordinate'] = final_df['y_coordinate'].astype(float)
final_df['latitude'] = final_df['latitude'].astype(float)
final_df['x_coordinate'] = final_df['x_coordinate'].astype(float)
final_df['ward'] = final_df['ward'].astype(int)
final_df['year'] = final_df['year'].astype(int)
final_df['domestic'] = final_df['domestic'].astype(int)
final_df['longitude'] = final_df['longitude'].astype(float)
final_df['beat'] = final_df['beat'].astype(int)
final_df['arrest'] = final_df['arrest'].astype(int)

# Convert date columns to text if needed
final_df['date'] = final_df['date'].astype(str)
final_df['updated_on'] = final_df['updated_on'].astype(str)

# Insert data into the 'Crime' table
final_df.to_sql('Crime', conn, if_exists='replace', index=False)

# Commit changes to the database
conn.commit()


In [36]:
# Convert data types for properties data
# Handle missing or None values in the 'mls_id' column by converting them to -1 (or any suitable default value)
all_properties['sold_price'] = all_properties['sold_price'].fillna(-1).astype(float)
all_properties['price_per_sqft'] = all_properties['price_per_sqft'].fillna(-1).astype(float)
all_properties['latitude'] = all_properties['latitude'].fillna(-1).astype(float)
all_properties['longitude'] = all_properties['longitude'].fillna(-1).astype(float)
all_properties['stories'] = all_properties['stories'].fillna(-1).astype(int)
all_properties['hoa_fee'] = all_properties['hoa_fee'].fillna(-1).astype(float)
all_properties['mls_id'] = all_properties['mls_id'].fillna(-1).astype(int)
all_properties['beds'] = all_properties['beds'].fillna(-1).astype(int)
all_properties['full_baths'] = all_properties['full_baths'].fillna(-1).astype(int)
all_properties['half_baths'] = all_properties['half_baths'].fillna(-1).astype(int)
all_properties['sqft'] = all_properties['sqft'].fillna(-1).astype(int)
all_properties['year_built'] = all_properties['year_built'].fillna(-1).astype(int)
all_properties['days_on_mls'] = all_properties['days_on_mls'].fillna(-1).astype(int)
all_properties['list_price'] = all_properties['list_price'].fillna(-1).astype(int)


# Insert data into the 'Property' table
all_properties.to_sql('Property', conn, if_exists='replace', index=False)

# Commit changes to the database
conn.commit()


In [37]:
# Transfer property data from all_properties DataFrame to the Property table in SQLite
all_properties.to_sql('Property', conn, if_exists='replace', index=False)

# Transfer crime data from final_df DataFrame to the Crime table in SQLite
final_df.to_sql('Crime', conn, if_exists='replace', index=False)

conn.commit()



In [38]:
db_path = 'chicago-crime-property.db'  

In [ ]:
# Connect to the SQLite database (replace 'your_database.db' with the actual path)
conn = sqlite3.connect('chicago-cirme-property.db')

# Create a cursor
cursor = conn.cursor()

# Execute any query

#